# Building ERDDAP Datasets

This notebook documents the process of creating XML fragments
for nowcast system run results files
for inclusion in `/opt/tomcat/content/erddap/datasets.xml`
on the `skookum` ERDDAP server instance.

The contents are a combination of:

* instructions for using the
`GenerateDatasetsXml.sh` and `DasDds.sh` tools found in the
`/opt/tomcat/webapps/erddap/WEB-INF/` directory
* instructions for forcing the server to update the datasets collection
via the `/results/erddap/flags/` directory
* code and metadata to transform the output of `GenerateDatasetsXml.sh`
into XML fragments that are ready for inclusion in `/opt/tomcat/content/erddap/datasets.xml`

In [1]:
from collections import OrderedDict

from lxml import etree

**NOTE**

The next cell mounts the `/results` filesystem on `skookum` locally.
It is intended for use if when this notebook is run on a laptop 
or other non-Waterhole machine that has `sshfs` installed 
and a mount point for `/results` available in its root filesystem.

Don't execute the cell if that doesn't describe your situation.

In [2]:
!sshfs skookum:/results /results

The `metadata` dictionary below contains information for dataset
attribute tags whose values need to be changed,
or that need to be added for all datasets.

The keys are the dataset attribute names.

The values are dicts containing a required `text` item
and perhaps an optional `after` item.

The value associated with the `text` key is the text content
for the attribute tag.

When present,
the value associated with the `after` key is the name
of the dataset attribute after which a new attribute tag
containing the `text` value is to be inserted.

In [3]:
metadata = OrderedDict([
    ('coverage_content_type', {
        'text': 'modelResult',
        'after': 'cdm_data_type',
    }),
    ('infoUrl', {
        'text': 
            'https://salishsea-meopar-tools.readthedocs.org/en/latest/results_server/index.html#salish-sea-model-results',
    }),
    ('institution', {'text': 'UBC EOAS'}),
    ('institution_fullname', {
        'text': 'Earth, Ocean & Atmospheric Sciences, University of British Columbia',
        'after': 'institution',
    }),
    ('license', {
        'text': '''The Salish Sea MEOPAR NEMO model results are copyright 2013-2016
by the Salish Sea MEOPAR Project Contributors and The University of British Columbia.

They are licensed under the Apache License, Version 2.0. http://www.apache.org/licenses/LICENSE-2.0''',
    }),
    ('project', {
        'text':'Salish Sea MEOPAR NEMO Model',
        'after': 'title',
    }),
    ('creator_name', {
        'text': 'Salish Sea MEOPAR Project Contributors',
        'after': 'project',
    }),
    ('creator_email', {
        'text': 'sallen@eos.ubc.ca',
        'after': 'creator_name',
    }),
    ('creator_url', {
        'text': 'https://salishsea-meopar-docs.readthedocs.org/',
        'after': 'creator_email',
    }),
    ('acknowledgement', {
        'text': 'MEOPAR, ONC, Compute Canada',
        'after': 'creator_url',
    }),
    ('drawLandMask', {
        'text': 'over',
        'after': 'acknowledgement',
    }),
])

The `datasets` dictionary below provides the content
for the dataset `title` and `summary` attributes.

The `title` attribute content appears in the the datasets list table
(among other places).
It should be `<`80 characters long,
and note that only the 1st 40 characters will appear in the table.

The `summary` atribute content appears
(among other places)
when a user hovers the cursor over the `?` icon beside the `title`
content in the datasets list table.
The text that is inserted into the `summary` attribute tag
by code later in this notebook is the
`title` content followed by the `summary` content,
separated by a blank line.

The keys of the `datasets` dict are the `datasetID` strings that
are used in many places by the ERDDAP server.
They are structured as follows:
* `ubc` to indicate that the dataset was produced at UBC
* `SSn` to indicate that the dataset is a product of the Salish Sea NEMO model
* a description of the dataset variables; e.g. `PointAtkinsonSSH` or `3DuVelocity`
* the time interval of values in the dataset; e.g. `15m`, `1h`, `1d`
* the dataset version; e.g. `V1`

So:
* `ubcSSnPointAtkinsonSSH15mV1` is the version 1 dataset of 15 minue averaged sea surface height values at Point Atkinson from `PointAtkinson.nc` output files
* `ubcSSn3DwVelocity1hV2` is the version 2 dataset of 1 hr averaged vertical (w) velocity values over the entire domain from `SalishSea_1h_*_grid_W.nc` output files
* `ubcSSnSurfaceTracers1dV1` is the version 1 dataset of daily averaged surface tracer values over the entire domain from `SalishSea_1d_*_grid_T.nc` output files

The dataset version part of the `datasetID` is used to indicate changes in the variables
contained in the dataset.
For example,
the transition from the `ubcSSn3DwVelocity1hV1` to the `ubcSSn3DwVelocity1hV2` dataset
occurred on 24-Jan-2016 when we started to output vertical eddy viscosity and diffusivity
values at the `w` grid points.

All datasets start at `V1` and their `summary` ends with a notation about the variables
that they contain; e.g.
```
v1: wVelocity variable
```
When the a dataset version is incremented a line describing the change is added
to the end of its `summary`; e.g.
```
v1: wVelocity variable
v2: Added eddy viscosity & diffusivity variables ve_eddy_visc & ve_eddy_diff
```

In [41]:
datasets = {
    'ubcSSnPointAtkinsonSSH15mV1': {
        'title': 'Nowcast, Point Atkinson, Sea Surface Height, 15min, v1',
        'summary': '''Sea surface height values averaged over 15 minute intervals from 
Salish Sea NEMO model nowcast runs. The values are calculated at the model grid point 
closest to the Point Atkinson tide gauge station on the north side of English Bay, 
near Vancouver, British Columbia.

v1: ssh variable''',        
    },
    'ubcSSn3DTracerFields1hV1': {
        'title': 'Nowcast, Salish Sea, 3d Tracer Fields, Hourly, v1',
        'summary': '''3d salinity and water temperature field values averaged over 1 hour intervals
from Salish Sea NEMO model nowcast runs. The values are calculated for the entire model grid
that includes the Strait of Juan de Fuca, the Strait of Georgia, Puget Sound,
and Johnstone Strait on the coasts of Washington State and British Columbia.

v1: salinity (practical) and temperature variables''',
        'fileNameRegex': '.*SalishSea_1h_\d{8}_\d{8}_grid_T\.nc$',
    },
    'ubcSSn3DuVelocity1hV1': {
        'title': 'Nowcast, Salish Sea, 3d u Velocity Field, Hourly, v1',
        'summary': '''3d zonal (u) component velocity field values averaged over 1 hour intervals
from Salish Sea NEMO model nowcast runs. The values are calculated for the entire model grid
that includes the Strait of Juan de Fuca, the Strait of Georgia, Puget Sound,
and Johnstone Strait on the coasts of Washington State and British Columbia.

v1: uVelocity variable''',
        'fileNameRegex': '.*SalishSea_1h_\d{8}_\d{8}_grid_U\.nc$',
    },
    
    'ubcSSn3DvVelocity1hV1': {
        'title': 'Nowcast, Salish Sea, 3d v Velocity Field, Hourly, v1',
        'summary': '''3d meridional (v) component velocity field values averaged over 1 hour intervals
from Salish Sea NEMO model nowcast runs. The values are calculated for the entire model grid
that includes the Strait of Juan de Fuca, the Strait of Georgia, Puget Sound,
and Johnstone Strait on the coasts of Washington State and British Columbia.

v1: vVelocity variable''',
        'fileNameRegex': '.*SalishSea_1h_\d{8}_\d{8}_grid_V\.nc$',
    },
    
    'ubcSSn3DwVelocity1hV1': {
        'title': 'Nowcast, Salish Sea, 3d w Velocity Field, Hourly, v1',
        'summary': '''3d vertical (w) component velocity field values averaged over 1 hour intervals
from Salish Sea NEMO model nowcast runs. The values are calculated for the entire model grid
that includes the Strait of Juan de Fuca, the Strait of Georgia, Puget Sound,
and Johnstone Strait on the coasts of Washington State and British Columbia.

v1: wVelocity variable''',
        'fileNameRegex': '.*SalishSea_1h_\d{8}_\d{8}_grid_W\.nc$',
    },
}
datasets['ubcSSn3DwVelocity1hV2'] = {
    'title': datasets['ubcSSn3DwVelocity1hV1']['title'].replace(', v1', ', v2'),
    'summary': datasets['ubcSSn3DwVelocity1hV1']['summary'] + '''
v2: Added eddy viscosity & diffusivity variables ve_eddy_visc & ve_eddy_diff''',
    'fileNameRegex': '.*SalishSea_1h_\d{8}_\d{8}_grid_W\.nc$',
}

The `dataset_vars` dictionary below is used to rename
variables from the often cryptic NEMO names to the names
that appear in the ERDDAP generated files and web content.

The keys are the NEMO variable names to replace.

The values are dicts that map the variable names to use in ERDDAP
to the `destinationName` attribute name.

In [34]:
dataset_vars = {
    'sossheig': {'destinationName': 'ssh'},
    'vosaline': {'destinationName': 'salinity'},
    'votemper': {'destinationName': 'temperature'},
    'vozocrtx': {'destinationName': 'uVelocity'},
    'vomecrty': {'destinationName': 'vVelocity'},
    'vovecrtz': {'destinationName': 'wVelocity'},
}

A couple of convenient functions to reduce code repetition:

In [6]:
def print_tree(root):
    """Display an XML tree fragment with indentation.
    """
    print(etree.tostring(root, pretty_print=True).decode('ascii'))

In [7]:
def find_att(root, att):
    """Return the dataset attribute element named att
    or raise a ValueError exception if it cannot be found.
    """
    e = root.find('.//att[@name="{}"]'.format(att))
    if e is None:
        raise ValueError('{} attribute element not found'.format(att))
    return e

Now we're ready to produce a dataset!!!

Use the `/opt/tomcat/webapps/erddap/WEB-INF/GenerateDatasetsXml.sh` script
generate the initial version of an XML fragment for a dataset:
```
$ cd /opt/tomcat/webapps/erddap/WEB-INF/
$ bash GenerateDatasetsXml.sh EDDGridFromNcFiles /results/SalishSea/nowcast/
```
The `EDDGridFromNcFiles` and `/results/SalishSea/nowcast/` arguments
tell the script which `EDDType` and what parent directory to use,
avoiding having to type those in answer to prompts.
Answer the remaining prompts,
for example:
```
File name regex (e.g., ".*\.nc") (default="")
? .*SalishSea_1h_\d{8}_\d{8}_grid_W\.nc$

Full file name of one file (default="")
? /results/SalishSea/nowcast/28jan16/SalishSea_1h_20160128_20160128_grid_W.nc

ReloadEveryNMinutes (e.g., 10080) (default="")
? 10080
```
Other examples of file name regex are:
* `.*PointAtkinson.nc$`
* `.*SalishSea_1d_\d{8}_\d{8}_grid_W\.nc$`

The output is written to `/results/erddap/logs/GenerateDatasetsXml.out`

Now, we:
* set the `datasetID` we want to use
* parse the output of `GenerateDatasetsXml.sh` into an XML tree data structure
* set the `datasetID` dataset attribute value
* re-set the `fileNameRegex` dataset attribute value because it looses its `\` characters during parsing(?)
* edit and add dataset attributes from the `metadata` dict
* set the `title` and `summary` dataset attributes from the `datasets` dict
* set the names of the grid `x` and `y` axis variables
* rename data variables as specified in the `dataset_vars` dict

In [33]:
def update_xml(root, datasetID, metadata, datasets, dataset_vars):
    root.attrib['datasetID'] = datasetID
    root.find('.//fileNameRegex').text = datasets[datasetID]['fileNameRegex']

    for att, info in metadata.items():
        e = etree.Element('att', name=att)
        e.text = info['text']
        try:
            root.find('.//att[@name="{}"]'.format(info['after'])).addnext(e)
        except KeyError:
            find_att(root, att).text = info['text']
        
    title = datasets[datasetID]['title']
    find_att(root, 'title').text = title
    find_att(root, 'summary').text = '{0}\n\n{1}'.format(title, datasets[datasetID]['summary'])

    for axis_name in root.findall('.//axisVariable/destinationName'):
        if axis_name.text in ('x', 'y'):
            axis_name.text = 'grid{}'.format(axis_name.text.upper())
        
    for var_name in root.findall('.//dataVariable/destinationName'):
        if var_name.text in dataset_vars:
            var_name.text = dataset_vars[var_name.text]['destinationName']

In [42]:
parser = etree.XMLParser(remove_blank_text=True)
tree = etree.parse('/results/erddap/logs/GenerateDatasetsXml.out', parser)
root = tree.getroot()

datasetID = 'ubcSSn3DwVelocity1hV2'

update_xml(root, datasetID, metadata, datasets, dataset_vars)

Inspect the resulting dataset XML fragment below and edit the dicts and
code cell above until it is what is required for the dataset:

In [43]:
print_tree(root)

<dataset type="EDDGridFromNcFiles" datasetID="ubcSSn3DwVelocity1hV2" active="true">
  <reloadEveryNMinutes>60</reloadEveryNMinutes>
  <updateEveryNMillis>10000</updateEveryNMillis>
  <fileDir>/results/SalishSea/nowcast/</fileDir>
  <recursive>true</recursive>
  <fileNameRegex>.*SalishSea_1h_\d{8}_\d{8}_grid_W\.nc$</fileNameRegex>
  <metadataFrom>last</metadataFrom>
  <matchAxisNDigits>20</matchAxisNDigits>
  <fileTableInMemory>false</fileTableInMemory>
  <accessibleViaFiles>false</accessibleViaFiles>
  <!-- sourceAttributes>
        <att name="Conventions">CF-1.1</att>
        <att name="file_name">SalishSea_1h_20160131_20160131_grid_W.nc</att>
        <att name="history">Sun Jan 31 12:03:50 2016: ncks -4 -L4 -O SalishSea_1h_20160131_20160131_grid_W.nc SalishSea_1h_20160131_20160131_grid_W.nc</att>
        <att name="NCO">4.4.2</att>
        <att name="production">An IPSL model</att>
        <att name="TimeStamp">31/01/2016 11:40:09 -0800</att>
    </sourceAttributes -->
  <addAttribut

In [17]:
axes = root.findall('.//axisVariable')

In [18]:
lon_axis = etree.Element('axisVariable')
source_name = etree.SubElement(lon_axis, 'sourceName')
source_name.text = 'nav_lon'
destination_name = etree.SubElement(lon_axis, 'destination_name')
destination_name.text = 'longitude'
axes[1] = lon_axis

In [19]:
print_tree(root)

<dataset type="EDDGridFromNcFiles" datasetID="ubcSSn3DwVelocity1hV1" active="true">
  <reloadEveryNMinutes>60</reloadEveryNMinutes>
  <updateEveryNMillis>10000</updateEveryNMillis>
  <fileDir>/results/SalishSea/nowcast/</fileDir>
  <recursive>true</recursive>
  <fileNameRegex>.*PointAtkinson.nc$</fileNameRegex>
  <metadataFrom>last</metadataFrom>
  <matchAxisNDigits>20</matchAxisNDigits>
  <fileTableInMemory>false</fileTableInMemory>
  <accessibleViaFiles>false</accessibleViaFiles>
  <!-- sourceAttributes>
        <att name="Conventions">CF-1.1</att>
        <att name="file_name">PointAtkinson.nc</att>
        <att name="production">An IPSL model</att>
        <att name="TimeStamp">2016-JAN-28 10:05:16 GMT-0800</att>
    </sourceAttributes -->
  <addAttributes>
    <att name="cdm_data_type">Grid</att>
    <att name="coverage_content_type">modelResult</att>
    <att name="Conventions">CF-1.6, COARDS, ACDD-1.3</att>
    <att name="infoUrl">https://salishsea-meopar-tools.readthedocs.org/e

Store the XML fragment for the dataset:

In [44]:
with open('/results/erddap_datasets_xml/{}.xml'.format(datasetID), 'wb') as f:
    f.write(etree.tostring(root, pretty_print=True))

Edit `/opt/tomcat/content/erddap/datasets.xml` to include the
XML fragment for the dataset that was stored by the abave cell.

Run `/opt/tomcast/webapps/erddap/WEB-INF/DasDds.sh` to:
* confirm that the XML fragment is correct
* create the `.das` and `.ddl` files for the dataset

`DasDds.hs` takes a `datasetID` as input from the command-line
and generates a ridiculous amount of output on the screen
and in `/results/erddap/logs/DasDds.log`
and `/results/erddap/logs/DasDds.out`.

If `DasDds.sh` finishes without errors and produces
metadata that looks sensible for the dataset,
create a flag file to signal the ERDDAP server process to load the dataset:
```
$ cd /results/erddap/flag/
$ touch <datasetID>
```